# Music Classification using Neural Networks
by: Team 22
***
This notebook will serve as the 'main' training & classification point of network. It will be kept as easy-to-read as possible by importing necessary pre-processing, data loaders, and other helper functions that do not need to be tuned with the network.

In [1]:
# import libraries
import torch
import matplotlib.pyplot as plt

# files and networks made by us
from DataProcessing import SplitTVT as tvt
from Networks.CRNN import CRNN
from Networks import Helpers as hp

In [2]:
pathSets = ["Data/trainSet.pkl", "Data/valSet.pkl", "Data/testSet.pkl"]  # path to datasets

In [3]:
# hyperparameters
bs = 64  # batch size
lr = 0.0001  # learning rate
epo = 30  # epochs

# hyperparameters for rnn
hidden_size = 10
num_layers = 1

In [4]:
trainLoader, valLoader, testLoader = tvt.load_data(pathSets[0], pathSets[1], pathSets[2], bs)

In [5]:
net = CRNN("crnn_1", hiddenSize=hidden_size, numLayers=num_layers)

In [6]:
hp.train(net, trainLoader, valLoader, lr, bs, epo)

Training Started...


AttributeError: 'list' object has no attribute 'labels'